In [ ]:
import numpy as np
import torchvision
import time
import os
import copy
import pdb
import time
import argparse

import sys
import cv2

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms

from dataloader import CocoDataset, CSVDataset, collater, Resizer, AspectRatioBasedSampler, Augmenter, UnNormalizer, Normalizer
from datetime import timedelta

In [3]:
assert torch.__version__.split('.')[1] == '4'

In [5]:
# 	parser = argparse.ArgumentParser(description='Simple training script for training a RetinaNet network.')

# 	parser.add_argument('--dataset', help='Dataset type, must be one of csv or coco.', default='coco')
# 	parser.add_argument('--coco_path', help='Path to COCO directory', default='coco')
# 	parser.add_argument('--csv_classes', help='Path to file containing class list (see readme)')
# 	parser.add_argument('--csv_val', help='Path to file containing validation annotations (optional, see readme)')

# 	parser.add_argument('--model', help='Path to model (.pt) file.', default='pretrained/coco_resnet_50_map_0_335.pt')
# 	parser.add_argument('--state_dict', help='Path to state dict (.pth) file.', default='pretrained/coco_resnet_50_map_0_335.pt')

# 	parser = parser.parse_args(args)
    
dataset = 'coco'
if dataset == 'coco':
    dataset_val = CocoDataset(dataset, set_name='val2017', transform=transforms.Compose([Normalizer(), Resizer()]))

model_path = 'pretrained/coco_resnet_50_map_0_335.pt'
retinanet = torch.load(model_path)

use_gpu = True

if use_gpu:
    retinanet = retinanet.cuda()

def draw_caption(image, box, caption):

    b = np.array(box).astype(int)
    cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 0), 2)
    cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1)

#     image_list = ['1.png', '2.png']
image_list = ['1.png']
for img in image_list:
    image = cv2.imread(img)
    total_time=0

    with torch.no_grad():
        retinanet.eval()

        image_2 = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))])

        image_2 = transform(image_2)
        image_2 = image_2.view((-1, image_2.size()[0],image_2.size()[1],image_2.size()[2]))
        st = time.time()
        scores, classification, transformed_anchors = retinanet(image_2.cuda().float())
        total_time += time.time()-st
        # print('Elapsed time: {}'.format(time.time()-st))
        idxs = np.where(scores>0.5)
        # print(torch.max(image_2),torch.min(image_2))
        # print(np.max(image),np.min(image))

        for j in range(idxs[0].shape[0]):
            bbox = transformed_anchors[idxs[0][j], :]
            x1 = int(bbox[0])
            y1 = int(bbox[1])
            x2 = int(bbox[2])
            y2 = int(bbox[3])
            label_name = dataset_val.labels[int(classification[idxs[0][j]])]
            draw_caption(image, (x1, y1, x2, y2), label_name)

            cv2.rectangle(image, (x1, y1), (x2, y2), color=(0, 0, 255), thickness=2)
            # print(label_name)

        # cv2.imwrite('final_vid/frame_{}.jpg'.format(count), image)

        cv2.imshow('Image',image)
        cv2.waitkey(0)
        cv2.destroyAllWindows()
    print("Total time taken is = {}".format(str(timedelta(seconds=total_time))))

loading annotations into memory...
Done (t=0.62s)
creating index...
index created!


ImportError: /home/users/a2pal/Work/ECE_285/pytorch-retinanet/lib/nms/_ext/nms/_nms.so: undefined symbol: PyInt_FromLong

In [6]:
print(sys.version)

3.5.2 (default, Nov 23 2017, 16:37:01) 
[GCC 5.4.0 20160609]
